In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

In [85]:

sub_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/sample_submission.csv')
train_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/train.csv')
test_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/test.csv')

In [86]:
train_df.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [87]:
train_df.isnull().any()

id                False
Sex               False
Length            False
Diameter          False
Height            False
Weight            False
Shucked Weight    False
Viscera Weight    False
Shell Weight      False
Age               False
dtype: bool

## handling the sex column

In [88]:
train_sex_col = pd.get_dummies(train_df['Sex'], prefix='Sex', prefix_sep='_', drop_first=True)
test_sex_col = pd.get_dummies(test_df['Sex'], prefix='Sex', prefix_sep='_', drop_first=True)

In [89]:
train_sex_col.head()

,Sex_I,Sex_M
0,1,0
1,1,0
2,0,1
3,0,0
4,1,0


In [90]:
train_df=pd.concat([train_df,train_sex_col],axis=1)
test_df=pd.concat([test_df,test_sex_col],axis=1)

    

In [91]:
train_df.drop(['id','Sex'],axis=1, inplace=True)
test_df.drop(['id','Sex'],axis=1, inplace=True)


In [92]:
train_df.head()

,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,Sex_I,Sex_M
0,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9,1,0
1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8,1,0
2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9,0,1
3,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11,0,0
4,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8,1,0


In [93]:
train_df.columns

Index(['Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight', 'Age', 'Sex_I', 'Sex_M'],
      dtype='object')

# Feature Enginnering

In [95]:
def generate_features(data):
  df_ = data.copy()
  df_['Shell Weight'] = np.where(df_['Shell Weight'] > df_['Weight'],df_['Weight'],df_['Shell Weight'])
  df_['Viscera Weight'] = np.where(df_['Viscera Weight'] > df_['Weight'],df_['Weight'],df_['Viscera Weight'])
  df_['Shucked Weight'] = np.where(df_['Shucked Weight'] > df_['Weight'],df_['Weight'],df_['Shucked Weight'])

  return df_
new_train = generate_features(train_df)
new_test = generate_features(test_df)

# Modeling

In [106]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from sklearn.svm import SVR

In [100]:
X=new_train.drop('Age', axis=1)
y=new_train['Age']

In [101]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=42)

In [102]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(59240, 9)
(59240,)
(14811, 9)
(14811,)


In [112]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping



# Define the deep learning model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1, validation_data=(X_test, y_test), callbacks=[early_stopping])


# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print("Mean absolute Error:", mae)

Epoch 1/100
1852/1852 [==============================] - 4s 2ms/step - loss: 5.7314 - val_loss: 5.8017
Epoch 2/100
1852/1852 [==============================] - 4s 2ms/step - loss: 4.5772 - val_loss: 4.3664
Epoch 3/100
1852/1852 [==============================] - 5s 3ms/step - loss: 4.5138 - val_loss: 4.7141
Epoch 4/100
1852/1852 [==============================] - 4s 2ms/step - loss: 4.4656 - val_loss: 4.3581
Epoch 5/100
1852/1852 [==============================] - 4s 2ms/step - loss: 4.4359 - val_loss: 4.3644
Epoch 6/100
1852/1852 [==============================] - 4s 2ms/step - loss: 4.4239 - val_loss: 4.4136
Epoch 7/100
1852/1852 [==============================] - 4s 2ms/step - loss: 4.4098 - val_loss: 4.7177
Epoch 8/100
1852/1852 [==============================] - 4s 2ms/step - loss: 4.4019 - val_loss: 4.3245
Epoch 9/100
1852/1852 [==============================] - 4s 2ms/step - loss: 4.4035 - val_loss: 4.3245
Epoch 10/100
1852/1852 [==============================] - 5s 3ms/step - l

# Prediction

**Based on the above result svr yeilds the better results**

In [113]:

# Make predictions on the scaled test data using the best model
y_test_pred = model.predict(test_df)

# Print the predicted values
print("Predicted Age for Test Data:")
print(y_test_pred)

1543/1543 [==============================] - 2s 1ms/step
Predicted Age for Test Data:
[[ 7.4146104]
 [ 7.910332 ]
 [10.790245 ]
 ...
 [12.785315 ]
 [10.502437 ]
 [12.357481 ]]


In [114]:
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/test.csv')

In [117]:
y_test_pred = np.squeeze(y_test_pred)

submission_df = pd.DataFrame({'id': test['id'], 'Age': y_test_pred})
submission_df.to_csv('submission.csv', index=False)

In [118]:
submission_df

,id,Age
0,74051,7.414610
1,74052,7.910332
2,74053,10.790245
3,74054,9.451137
4,74055,7.565471
...,...,...
49363,123414,9.537012
49364,123415,7.928013
49365,123416,12.785315
49366,123417,10.502437
